In [ ]:
%matplotlib inline

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

import os

import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow import keras

from keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import Precision, Recall
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from keras.preprocessing import image

import matplotlib.pyplot as plt

from sklearn.utils.class_weight import compute_class_weight

## Load Dataset

In [ ]:
train_dir = './ds/train/'
test_dir = './ds/test/'

classes = [os.path.basename(x[0]) for x in os.walk("./ds/train") if x[0]][1:]
num_classes = len(classes)
print(classes)

In [ ]:
def load_data(path):
    data = []
    for c, category in enumerate(classes):
        images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(path + category) for f in filenames]

        for img_path in images:
            # Note that we apply the same preprocessing and target images of size 224x224
            # in order to match the setup of original EfficientNet model
            img = image.load_img(img_path, target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)

            data.append({'x' : np.array(x[0]), 'y' : c})

    return data


data_train_val = load_data(train_dir)
data_test = load_data(test_dir)

In [ ]:
X_train_val, y_train_val = np.array([t["x"] for t in data_train_val]), [t["y"] for t in data_train_val]
X_test, y_test = np.array([t["x"] for t in data_test]), [t["y"] for t in data_test]

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=42, test_size=0.2)

# Convert classes to one-hot vectors
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

print('Train images:', len(y_train), y_train.shape)
print('Val images:', len(y_val), y_val.shape)
print('Test images:', len(y_test), y_test.shape)

## Data Enhancement

#### Balance classes

In [ ]:
# Compute class weights to let the CNN pay attention to less frequent classes when fitting the model
y_integers = np.argmax(y_train, axis=1)
class_weights = compute_class_weight(class_weight = "balanced",
                                     classes = np.unique(y_integers),
                                     y = y_integers)
class_weights = dict(zip(np.unique(y_integers), class_weights))

#### Generate artificial images using shift, rotation, flip

In [ ]:
# Enrich train data set by rotation, flip, shift
gen_train = ImageDataGenerator(rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   data_format='channels_last',
                                   preprocessing_function=preprocess_input)

# No augmentation for test and validation sets
gen_val_test = ImageDataGenerator(rotation_range=0,
                                   width_shift_range=0.0,
                                   height_shift_range=0.0,
                                   horizontal_flip=False,
                                   data_format='channels_last',
                                   preprocessing_function=preprocess_input)

# Generate iterators for train/val/test sets
train_iter = gen_train.flow(X_train, y_train, batch_size=32)
val_iter = gen_val_test.flow(X_val, y_val, batch_size=32)
test_iter = gen_val_test.flow(X_test, y_test, batch_size=32)


## Model Creation and Training

In [ ]:
def get_model():
    model = Sequential()
    print("Input dimensions:", X_train.shape[1:])

    model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))

    model.add(Dropout(0.5))

    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # model.summary()

    return model


In [ ]:
baseline_model = get_model()

baseline_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', Precision(), Recall()])

In [ ]:
# Create checkpoints during training
filepath = './checkpoints/baseline_v2/baseline_v2_{epoch:02d}_{val_accuracy:.2f}.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = baseline_model.fit(train_iter,
                    batch_size=32,
                    epochs=128,
                    validation_data=val_iter,
                    callbacks=callbacks_list,
                    class_weight=class_weights)

## Evaluation

In [ ]:
fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(history.history["val_loss"])
ax.set_title("validation loss")
ax.set_xlabel("epochs")

ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_accuracy"])
ax2.set_title("validation accuracy")
ax2.set_xlabel("epochs")
ax2.set_ylim(0, 1)

plt.show()